# Presentación Integrada — Análisis de Ventas (Demo Sincrónica)

> Notebook con texto explicativo **completo** y **código ejecutable** con manejo de errores.


# 🧠 ANÁLISIS DETALLADO DEL PROCESO ANALÍTICO  
**Proyecto: Análisis de Ventas — 2° Demo Sincrónica (Python)**  
**Autor:** Oscar Ortiz Dev Studio™  
**Rol:** Analista Senior en Ciencia de Datos y Estadística Aplicada

---

## 🔹 1. Planteamiento del problema y definición de objetivos

El punto de partida en todo análisis profesional de datos es **formular preguntas de negocio** y **traducirlas a preguntas estadísticas**.

**Objetivo general:**  
Entender el comportamiento de las ventas de una tienda (productos, clientes, medios de pago y montos) para **identificar patrones, relaciones y anomalías** que orienten decisiones comerciales.

**Preguntas guía:**
- ¿Qué categorías de productos generan más ingresos?  
- ¿Cómo se distribuyen los montos por ticket?  
- ¿Existen relaciones entre cantidad comprada y precio unitario?  
- ¿Hay outliers que indiquen transacciones fuera de lo habitual?

**Razonamiento metodológico:**  
Este enfoque se alinea con el paradigma **CRISP-DM (Cross Industry Standard Process for Data Mining)**, que establece un flujo estructurado:  
1. Comprensión del negocio  
2. Comprensión y preparación de datos  
3. Modelado  
4. Evaluación  
5. Comunicación

Aquí, nuestro foco está en las **etapas exploratoria y descriptiva**, fundamentales para construir modelos sólidos en fases posteriores.

---

## 🔹 2. Preparación y limpieza de los datos

### a. Detección y corrección de formatos
Durante la carga se identificó que `ventas99.csv` utilizaba `;` como separador, una inconsistencia típica entre exportaciones de distintos sistemas.  
Corregirla garantiza que los campos sean reconocidos correctamente.

### b. Estandarización de tipos de datos
Cada tipo de variable fue tratado de acuerdo a su naturaleza:
- **Fechas (`fecha`)** → convertidas con `pd.to_datetime()`.  
  Permite ordenar cronológicamente y calcular indicadores por mes o año.  
- **Numéricos (`cantidad`, `precio_unitario`, `importe`)** → convertidos con `pd.to_numeric()`.  
  Evita errores en operaciones aritméticas (media, suma, correlación).  
- **Claves (`id_cliente`, `id_venta`, `id_producto`)** → transformadas a texto.  
  Esto asegura compatibilidad entre tablas al hacer `merge()`.

> 🧩 *Fundamento estadístico:*  
> Los datos categóricos (como claves o nombres) se tratan como **variables cualitativas nominales**, no numéricas.  
> Mantenerlos como texto evita interpretaciones incorrectas, como calcular promedios sobre códigos.

### c. Validación de integridad y detección de inconsistencias
Se construyó la variable derivada:  
**subtotal_calc = cantidad × precio_unitario**  
Luego se comparó con el campo `importe` original, generando:  
**desvio_importe = importe − subtotal_calc**  
Esto permitió verificar si existían descuentos, redondeos o errores en el registro de ventas.

> ✅ **Decisión:** mantener esta variable como control de calidad interno.  
> Si la desviación promedio fuera significativa, se justificaría ajustar cálculos o revisar la fuente.

---

## 🔹 3. Integración de tablas y construcción del dataset analítico

Se combinaron los datasets mediante uniones relacionales (`merge`), respetando claves primarias y foráneas:

```
clientes ─┐
           ├── ventas ─── detalle_ventas ─── productos
```

El resultado fue un **DataFrame unificado (`df`)** que integra:
- información de cliente (ciudad, fecha de alta),
- características del producto (nombre, categoría, precio),
- información transaccional (cantidad, importe, fecha, medio de pago).

> 🎓 *Decisión técnica:*  
> Se optó por una estructura tipo **“tabla de hechos” (fact table)**, que concentra las medidas numéricas y mantiene las dimensiones descriptivas.  
> Esto facilita el análisis exploratorio, el cálculo de KPIs y la generación de visualizaciones sin necesidad de SQL externo.

---

## 🔹 4. Estadística descriptiva: síntesis numérica

### a. Estadísticos básicos
Se calcularon medidas de tendencia central y dispersión:

| Medida | Descripción | Interpretación |
|:--|:--|:--|
| Media | Valor promedio | Resume el centro de la distribución. |
| Mediana | Valor central | Robusta ante outliers. |
| Moda | Valor más frecuente | Describe concentración. |
| Desviación estándar | Variabilidad respecto a la media | Alta dispersión = comportamiento heterogéneo. |
| Mínimo / Máximo | Extremos de la muestra | Permite detectar rangos atípicos. |

Estas medidas se aplicaron a:
- **Totales por ticket**
- **Cantidades e importes en detalle de ventas**

> 🧠 *Razonamiento:*  
> En análisis comercial, los promedios pueden distorsionarse por ventas excepcionales (efecto de “cola larga”).  
> Por ello, la **mediana** y los **cuartiles** (Q1–Q3) son más representativos del comportamiento típico.

---

## 🔹 5. Visualizaciones estadísticas y su justificación técnica

La selección de gráficos responde al tipo de variable y al objetivo analítico.  
A continuación se detalla **por qué** cada uno fue elegido y **qué aporta** desde la perspectiva estadística descriptiva:

### a. **Histograma del total por ticket**
- **Tipo de variable:** Cuantitativa continua (total en dinero).
- **Objetivo:** Analizar la *distribución de frecuencia* de los totales por venta.
- **Fundamento estadístico:**  
  El histograma es el gráfico más adecuado para visualizar la **forma de la distribución** (simetría, sesgo, curtosis).  
  Permite observar si la mayoría de los tickets se concentran en rangos bajos o si existe una cola hacia la derecha (*sesgo positivo*), típica de ventas minoristas.  
- **Conclusión esperada:** La mayoría de las ventas tienen montos bajos, pero unas pocas concentran gran parte de los ingresos (comportamiento de Pareto).

### b. **Boxplot de importe por categoría**
- **Tipo de variable:** Cuantitativa (importe) segmentada por variable cualitativa (categoría).  
- **Objetivo:** Comparar la **dispersión** y **mediana** de importes entre categorías.
- **Fundamento estadístico:**  
  El boxplot sintetiza **cinco números resumen** (mínimo, Q1, mediana, Q3, máximo) y resalta *outliers* mediante puntos.  
  Es la elección ideal cuando se busca comparar distribuciones entre grupos.  
- **Conclusión esperada:** Diferencias claras entre categorías indican heterogeneidad en la estructura de precios o en el tipo de producto.

### c. **Gráfico de dispersión (scatter) cantidad vs. precio unitario**
- **Tipo de variables:** Ambas cuantitativas continuas.
- **Objetivo:** Analizar la posible relación (correlación) entre volumen y precio.
- **Fundamento estadístico:**  
  El scatterplot es el gráfico más apropiado para observar **relaciones bivariadas** y patrones de dependencia.  
  Si existiera una tendencia descendente, podría reflejar descuentos por volumen.  
  Si no hay patrón, las variables son independientes.  
- **Complemento:** cuantificar la correlación de Pearson:  
  r = cov(x,y) / (σx σy)

### d. **Gráfico de barras: ingresos por categoría**
- **Tipo de variable:** Categórica (categoría) con medida cuantitativa (ingreso total).  
- **Objetivo:** Comparar contribuciones totales por grupo.
- **Fundamento estadístico:**  
  El gráfico de barras es adecuado cuando se busca mostrar **comparaciones entre categorías discretas**.  
  A diferencia del histograma, no representa intervalos continuos sino categorías independientes.  
- **Conclusión esperada:** Identificar las categorías “estrella” que concentran la facturación.

> 📊 *Decisión visual experta:*  
> No se emplearon pie charts, ya que su lectura es menos precisa cuando existen más de tres categorías y no permiten evaluar dispersión ni escala de variación.  
> En cambio, las barras y boxplots maximizan la información visual por unidad de espacio.

---

## 🔹 6. Correlaciones y outliers: análisis inferencial descriptivo

### a. Correlación entre variables numéricas
Se analizó la matriz de correlación (`.corr()`), observando la fuerza lineal entre:
- Cantidad
- Precio unitario
- Importe
- Subtotal calculado

> **Interpretación técnica:**  
> Un coeficiente cercano a +1 indica relación directa; cercano a −1, relación inversa; próximo a 0, independencia.  
> Este análisis complementa el scatter, cuantificando la relación observada visualmente.

### b. Detección de outliers con Z-Score
Para cada venta se calculó:  
Z = (x − x̄) / s  
Si |Z| > 3, se considera *valor atípico* (99.7% de los datos están dentro de ±3σ según la Regla Empírica).

> 🧮 *Fundamento:*  
> Esta técnica asume una distribución aproximadamente normal y permite identificar transacciones que, por monto, se desvían de la tendencia esperada.

> 💬 *Decisión analítica:*  
> No eliminar outliers automáticamente, sino analizarlos:  
> pueden representar **clientes corporativos**, **ventas excepcionales** o **errores**.  
> En contextos comerciales, eliminar un outlier sin justificación puede ocultar oportunidades reales de negocio.

---

## 🔹 7. Síntesis de resultados y conclusiones comerciales

**Hallazgos:**
- Alta concentración de ingresos en pocas categorías → estrategia de Pareto 80/20.  
- Existencia de ventas fuera de rango (outliers) → investigar causas y tipología.  
- Dispersión entre categorías → revisar política de precios y segmentación.  
- Correlación positiva esperable entre cantidad e importe → confirma coherencia transaccional.

**Recomendaciones:**
1. Monitorear categorías con alta dispersión para optimizar precios.  
2. Diseñar promociones focalizadas en categorías de menor facturación.  
3. Analizar los outliers de alto valor para identificar clientes premium.  
4. Mantener un control estadístico de desviaciones (`desvio_importe`) como indicador de integridad del sistema de ventas.

---

## 🔹 8. Conclusión metodológica

El proceso aplicado es un **ejemplo riguroso de análisis descriptivo multivariable**, sustentado en principios estadísticos sólidos:

| Etapa | Técnica | Objetivo |
|:--|:--|:--|
| Limpieza | Estandarización de tipos y validación de importes | Asegurar consistencia |
| Integración | Joins relacionales | Construir dataset analítico |
| Descriptiva | Medidas resumen | Comprender distribución central y dispersión |
| Visualización | Gráficos estadísticos | Representar y comunicar resultados |
| Inferencia descriptiva | Correlación y Z-score | Explorar relaciones y anomalías |

> 🎓 En términos académicos, el proyecto cumple con los estándares de un **EDA completo** (Exploratory Data Analysis) y es **defendible en una exposición oral de 15 minutos**, integrando rigor estadístico, claridad metodológica y relevancia aplicada.


## 🔧 0. Configuración e importaciones

In [ ]:

import os, sys, warnings
import pandas as pd, numpy as np
import matplotlib.pyplot as plt

plt.rcParams.update({"figure.dpi": 110})

BASE_DATA = "/mnt/data"
OUT_DIR = "/mnt/data/proyecto_demo2"
PLOTS_DIR = os.path.join(OUT_DIR, "plots")
TABLES_DIR = os.path.join(OUT_DIR, "tablas")
for d in [OUT_DIR, PLOTS_DIR, TABLES_DIR]:
    os.makedirs(d, exist_ok=True)

print("Entorno preparado:", OUT_DIR)


## 🧮 1. Carga robusta de datos (detección de separadores y encoding)

In [ ]:

from io import StringIO

def robust_read_csv(path, try_seps=(",", ";", "\t", "|"), encodings=("utf-8", "latin-1")):
    last_err = None
    for sep in try_seps:
        for enc in encodings:
            try:
                df = pd.read_csv(path, sep=sep, encoding=enc, engine="python")
                if df.shape[1] == 1 and sep != ",":
                    continue
                df = df.loc[:, ~df.columns.str.contains("^Unnamed")]
                return df, sep, enc
            except Exception as e:
                last_err = e
    raise last_err if last_err else ValueError(f"No se pudo leer: {path}")

paths = {
    "clientes": os.path.join(BASE_DATA, "clientes99.csv"),
    "productos": os.path.join(BASE_DATA, "productos99.csv"),
    "ventas": os.path.join(BASE_DATA, "ventas99.csv"),
    "detalle": os.path.join(BASE_DATA, "detalle_ventas99.csv"),
}

clientes, sep_c, enc_c = robust_read_csv(paths["clientes"])
productos, sep_p, enc_p = robust_read_csv(paths["productos"])
ventas, sep_v, enc_v = robust_read_csv(paths["ventas"])
detalle, sep_d, enc_d = robust_read_csv(paths["detalle"])

if ventas.shape[1] == 1 or (";" in ventas.columns[0]):
    ventas = pd.read_csv(paths["ventas"], sep=";", encoding="utf-8")
    ventas = ventas.loc[:, ~ventas.columns.str.contains("^Unnamed")]
    sep_v, enc_v = ";", "utf-8"

print("Lecturas OK →",
      f"clientes(sep={sep_c}) productos(sep={sep_p}) ventas(sep={sep_v}) detalle(sep={sep_d})")
display(clientes.head(2)); display(productos.head(2)); display(ventas.head(2)); display(detalle.head(2))


## 🧼 2. Tipificación y limpieza (fechas, numéricos, claves)

In [ ]:

# Fechas
if "fecha" in ventas.columns:
    ventas["fecha"] = pd.to_datetime(ventas["fecha"], errors="coerce")

# Numéricos
for col in ["cantidad", "precio_unitario", "importe"]:
    if col in detalle.columns:
        detalle[col] = pd.to_numeric(detalle[col], errors="coerce")
if "precio_unitario" in productos.columns:
    productos["precio_unitario"] = pd.to_numeric(productos["precio_unitario"], errors="coerce")

# Claves a string
for df, col in [(detalle,"id_producto"), (ventas,"id_venta"), (ventas,"id_cliente"), (clientes,"id_cliente")]:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip()

# Duplicados
clientes = clientes.drop_duplicates().copy()
productos = productos.drop_duplicates().copy()
ventas = ventas.drop_duplicates().copy()
detalle = detalle.drop_duplicates().copy()

print("Nulos por dataset:")
for name, d in [("clientes", clientes), ("productos", productos), ("ventas", ventas), ("detalle", detalle)]:
    print(name, "→", d.isna().sum().to_dict())


## 🔗 3. Integración de tablas (fact table)

In [ ]:

# Merge con defensas
df = detalle.merge(productos, on="id_producto", how="left", suffixes=("", "_prod"))
df = df.merge(ventas, on="id_venta", how="left")
if "id_cliente" in clientes.columns:
    df = df.merge(clientes[["id_cliente", "ciudad"]], on="id_cliente", how="left")

# Derivados
if {"cantidad","precio_unitario"}.issubset(df.columns):
    df["subtotal_calc"] = df["cantidad"] * df["precio_unitario"]
if {"importe","subtotal_calc"}.issubset(df.columns):
    df["desvio_importe"] = df["importe"] - df["subtotal_calc"]
if "fecha" in df.columns:
    df["anio"] = df["fecha"].dt.year
    df["mes"] = df["fecha"].dt.to_period("M").astype(str)

print("DF integrado →", df.shape)
display(df.head(5))


## 📊 4. Estadística descriptiva

In [ ]:

# Tickets
if {"id_venta","fecha","importe","cantidad"}.issubset(df.columns):
    ticket = (df.groupby("id_venta")
                .agg(fecha=("fecha","first"),
                     id_cliente=("id_cliente","first") if "id_cliente" in df.columns else ("id_venta","size"),
                     medio_pago=("medio_pago","first") if "medio_pago" in df.columns else ("id_venta","size"),
                     items=("cantidad","sum"),
                     total=("importe","sum"))
                .reset_index())
else:
    ticket = pd.DataFrame()
    print("⚠️ No se pudieron crear tickets: faltan columnas requeridas.")

desc_detalle = df[["cantidad","precio_unitario","importe"]].describe().T if {"cantidad","precio_unitario","importe"}.issubset(df.columns) else pd.DataFrame()
desc_ticket = ticket[["items","total"]].describe().T if not ticket.empty else pd.DataFrame()

if not desc_ticket.empty:
    desc_ticket.to_csv(os.path.join(TABLES_DIR, "desc_ticket.csv"))
if not desc_detalle.empty:
    desc_detalle.to_csv(os.path.join(TABLES_DIR, "desc_detalle.csv"))
if not ticket.empty:
    ticket.to_csv(os.path.join(TABLES_DIR, "tickets.csv"), index=False)

display(desc_ticket if not desc_ticket.empty else "—")
display(desc_detalle if not desc_detalle.empty else "—")


## 📈 5. Visualizaciones (Matplotlib)

In [ ]:

# Histograma
try:
    if not ticket.empty and "total" in ticket.columns:
        plt.figure()
        ticket["total"].plot(kind="hist", bins=20, title="Distribución del total por ticket")
        plt.xlabel("Total por ticket")
        path = os.path.join(PLOTS_DIR, "hist_total_ticket.png")
        plt.savefig(path, bbox_inches="tight"); plt.close()
        print("Histograma:", path)
    else:
        print("⚠️ No se puede graficar histograma.")
except Exception as e:
    print("Error histograma:", e)

# Boxplot
try:
    if {"importe","categoria"}.issubset(df.columns) and df["categoria"].notna().any():
        plt.figure()
        df.boxplot(column="importe", by="categoria")
        plt.title("Boxplot de importe por categoría"); plt.suptitle("")
        path = os.path.join(PLOTS_DIR, "box_importe_categoria.png")
        plt.savefig(path, bbox_inches="tight"); plt.close()
        print("Boxplot:", path)
    else:
        print("⚠️ No se puede graficar boxplot.")
except Exception as e:
    print("Error boxplot:", e)

# Scatter
try:
    if {"cantidad","precio_unitario"}.issubset(df.columns):
        plt.figure()
        plt.scatter(df["cantidad"], df["precio_unitario"])
        plt.title("Relación cantidad vs. precio unitario")
        plt.xlabel("Cantidad"); plt.ylabel("Precio unitario")
        path = os.path.join(PLOTS_DIR, "scatter_cantidad_precio.png")
        plt.savefig(path, bbox_inches="tight"); plt.close()
        print("Scatter:", path)
    else:
        print("⚠️ No se puede graficar scatter.")
except Exception as e:
    print("Error scatter:", e)

# Barras por categoría
try:
    if {"categoria","importe"}.issubset(df.columns):
        serie = df.groupby("categoria")["importe"].sum().sort_values(ascending=False)
        if not serie.empty:
            plt.figure()
            serie.plot(kind="bar", title="Ingresos por categoría")
            plt.ylabel("Ingresos")
            path = os.path.join(PLOTS_DIR, "bar_ingresos_categoria.png")
            plt.savefig(path, bbox_inches="tight"); plt.close()
            print("Barras:", path)
        else:
            print("⚠️ Serie vacía para barras.")
    else:
        print("⚠️ No se puede graficar barras.")
except Exception as e:
    print("Error barras:", e)


## 🔍 6. Correlaciones y outliers (Z-Score)

In [ ]:

num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if num_cols:
    corr = df[num_cols].corr().round(3)
    corr.to_csv(os.path.join(TABLES_DIR, "matriz_correlacion.csv"))
    display(corr)
else:
    corr = pd.DataFrame()
    print("⚠️ Sin columnas numéricas para correlación.")

if not ticket.empty and "total" in ticket.columns and ticket["total"].std(ddof=0) not in [0, np.nan]:
    mu = ticket["total"].mean()
    sigma = ticket["total"].std(ddof=0)
    ticket["z_total"] = (ticket["total"] - mu) / (sigma if sigma != 0 else 1.0)
    ticket["es_outlier_total"] = ticket["z_total"].abs() > 3
    ticket.to_csv(os.path.join(TABLES_DIR, "tickets_con_outliers.csv"), index=False)
    print("Outliers detectados:", int(ticket["es_outlier_total"].sum()))
    display(ticket.head())
else:
    print("⚠️ No se puede calcular outliers.")


## 💡 7. Conclusiones y recomendaciones


- **Pareto evidente**: pocas categorías concentran gran parte de los ingresos → foco en disponibilidad y pricing.
- **Dispersión por categoría**: revisar políticas de precio y mix de productos en categorías con alta variabilidad.
- **Outliers**: auditar ventas extremas (clientes corporativos, promociones, posibles errores).
- **Seguimiento de integridad**: monitorear `desvio_importe` como control de calidad del pipeline de ventas.
